# Logistic Regression using PyTorch

In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

/home/faris/miniconda3/envs/cudalab/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Pipeline
  - Data:
      - Load dataset
      - DataLoader
      - Visualize dataset samples
  - Design Model:
      - Model Architecture
      - Forward Pass
  - Define Loss Function
  - Define Optimizer
  - Train:
      - Forward Pass (compute activations and loss)
      - Backward Pass (compute gradients)
      - Update Weights

## Prepare dataset

In [2]:
bc = datasets.load_breast_cancer()
X, y = bc.data, bc.target

In [3]:
X.shape

(569, 30)

In [4]:
y.shape

(569,)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

In [6]:
# scale
sc = StandardScaler()
X_train = sc.fit_transform(X_train) # find mean and std from X_train and transform/scale it
X_test = sc.transform(X_test) # scale with same parameters that you found with training data

In [7]:
X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

In [8]:
y_train.shape

torch.Size([455])

In [9]:
y_train = y_train.view(-1,1)
y_test = y_test.view(-1,1)

In [10]:
y_train.shape

torch.Size([455, 1])

## Model

In [15]:
in_features = X_train.shape[1]
out_features = y_train.shape[1]

In [21]:
class LogisticRegression(nn.Module):
    def __init__(self, in_features, out_features):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(in_features, out_features)
        self.activation = torch.nn.Sigmoid()

    def forward(self, x):
        #y_pred = torch.sigmoid(self.linear(x))
        #y_pred = self.activation(self.linear(x))
        logits = self.linear(x)
        y_pred = self.activation(logits)
        return y_pred

In [22]:
model = LogisticRegression(in_features, out_features)

In [23]:
model

LogisticRegression(
  (linear): Linear(in_features=30, out_features=1, bias=True)
  (activation): Sigmoid()
)

## Loss and Optimizer

In [14]:
criterion = nn.BCELoss()

In [15]:
learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

## Train

In [16]:
num_epochs = 100

In [17]:
for epoch in range(num_epochs):
    # Forward pass and loss
    y_pred = model(X_train)
    loss = criterion(y_pred, y_train)

    # Backward pass and update
    loss.backward()
    # Update weights
    optimizer.step()

    # zero grad before new step
    optimizer.zero_grad()

## Evaluate 1

In [18]:
y_test_pred = model(X_test)

In [19]:
type(y_test_pred)

torch.Tensor

In [20]:
y_test_pred_numpy = y_test_pred.detach().numpy()
y_test_pred_numpy = np.round(y_test_pred_numpy)

In [21]:
type(y_test_pred_numpy)

numpy.ndarray

In [22]:
y_test_numpy = y_test.numpy()

In [23]:
type(y_test_numpy)

numpy.ndarray

In [24]:
acc = (np.mean(y_test_pred_numpy == y_test_numpy))

In [25]:
print(f'accuracy: {acc.item():.4f}')

accuracy: 0.9298


## Evaluate 2

In [26]:
y_test_pred

tensor([[7.0787e-01],
        [9.4454e-01],
        [4.7695e-01],
        [8.4165e-01],
        [8.3959e-01],
        [5.5891e-01],
        [4.4336e-01],
        [9.2934e-01],
        [5.8180e-02],
        [7.4942e-02],
        [6.5387e-02],
        [6.8906e-01],
        [7.9179e-01],
        [6.2460e-01],
        [8.3095e-01],
        [3.5258e-03],
        [9.2198e-01],
        [9.4011e-01],
        [7.9232e-01],
        [2.4125e-01],
        [7.6829e-01],
        [2.9794e-01],
        [1.3574e-02],
        [6.1392e-03],
        [7.0141e-02],
        [7.5728e-01],
        [4.0058e-01],
        [8.8010e-01],
        [9.5119e-01],
        [9.1815e-01],
        [8.2439e-01],
        [8.8440e-01],
        [6.2271e-02],
        [6.5914e-01],
        [7.5912e-01],
        [5.9562e-01],
        [9.1733e-01],
        [2.6754e-01],
        [6.3329e-01],
        [8.8901e-01],
        [1.2902e-01],
        [8.1266e-01],
        [9.3420e-02],
        [6.9066e-01],
        [7.0768e-01],
        [9

In [27]:
with torch.no_grad():
    y_predicted = model(X_test)
    y_predicted_cls = y_predicted.round()
    acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(f'accuracy: {acc.item():.4f}')

accuracy: 0.9298


In [28]:
y_predicted

tensor([[7.0787e-01],
        [9.4454e-01],
        [4.7695e-01],
        [8.4165e-01],
        [8.3959e-01],
        [5.5891e-01],
        [4.4336e-01],
        [9.2934e-01],
        [5.8180e-02],
        [7.4942e-02],
        [6.5387e-02],
        [6.8906e-01],
        [7.9179e-01],
        [6.2460e-01],
        [8.3095e-01],
        [3.5258e-03],
        [9.2198e-01],
        [9.4011e-01],
        [7.9232e-01],
        [2.4125e-01],
        [7.6829e-01],
        [2.9794e-01],
        [1.3574e-02],
        [6.1392e-03],
        [7.0141e-02],
        [7.5728e-01],
        [4.0058e-01],
        [8.8010e-01],
        [9.5119e-01],
        [9.1815e-01],
        [8.2439e-01],
        [8.8440e-01],
        [6.2271e-02],
        [6.5914e-01],
        [7.5912e-01],
        [5.9562e-01],
        [9.1733e-01],
        [2.6754e-01],
        [6.3329e-01],
        [8.8901e-01],
        [1.2902e-01],
        [8.1266e-01],
        [9.3420e-02],
        [6.9066e-01],
        [7.0768e-01],
        [9